# Laboratorio 6

## Autores
- Luis Santos
- José Miguel González
- Carol Arévalo
- Stefano Aragoni

### Task 1

**1. ¿Qué es Prioritized sweeping para ambientes determinísticos?**

Prioritized sweeping es un algoritmo de planificación que se utiliza para encontrar la política óptima en un ambiente determinístico. Este algoritmo se basa en la idea de que las transiciones que tienen un mayor impacto en la política óptima son las que se deben priorizar. Para ello, se utiliza una cola de prioridad que se va actualizando a medida que se van explorando las transiciones del ambiente. De esta forma, se garantiza que las transiciones más importantes se exploren primero, lo que permite encontrar la política óptima de forma más eficiente.


**2. ¿Qué es Trajectory Sampling?**

Trajectory Sampling es un método de aprendizaje por refuerzo que se utiliza para estimar la función de valor de un estado a partir de las trayectorias de un agente en un ambiente. En este método, se generan múltiples trayectorias de un agente en el ambiente y se utilizan para estimar la función de valor de los estados visitados por el agente. De esta forma, se obtiene una estimación de la función de valor que se puede utilizar para mejorar la política del agente.


**3. ¿Qué es Upper Confidence Bounds para Árboles (UCT por sus siglas en inglés)?**

Upper Confidence Bounds para Árboles (UCT) es un algoritmo de búsqueda que se utiliza para encontrar la mejor acción a tomar en un árbol de búsqueda. Este algoritmo se basa en la idea de que se deben explorar las acciones que tienen un mayor potencial de mejorar la política del agente. Para ello, se utiliza una función de valor que combina la estimación de la recompensa de una acción con la incertidumbre asociada a esa estimación. De esta forma, se garantiza que se exploren las acciones que tienen un mayor potencial de mejorar la política del agente.

### Task 2

En este laboratorio, compararán el rendimiento de Dyna-Q+ y MCTS, dos de los algoritmos que vimos en clase, utilizando el entorno de FrozenLake-v1 de la biblioteca Gymnasium. Analizará y graficará las recompensas por episodio y responderá las preguntas que aparecen al final para asegurar su comprensión de los algoritmos.

#### Importar librerías

In [108]:
import numpy as np
import gym
import random
from math import sqrt, log

#### Implementación de MCTS

In [109]:
class Node:
    def __init__(self, state, parent=None):
        self.state = state
        self.parent = parent
        self.children = []
        self.visits = 0
        self.wins = 0

    def AddChild(self, child_state):
        child_node = Node(child_state, parent=self)
        self.children.append(child_node)
        return child_node

    def UCB1(self, exploration_weight=1.41):
        if self.visits == 0:
            return float('inf')
        return (self.wins / self.visits) + exploration_weight * sqrt(log(self.parent.visits) / self.visits)

class MCTS:
    def __init__(self, env, max_iterations=1000):
        self.env = env
        self.max_iterations = max_iterations

    def Search(self, root):
        for _ in range(self.max_iterations):
            node = self.Select(root)
            reward = self.Simulate(node.state)
            self.Backpropagate(node, reward)

    def Select(self, node):
        while node.children:
            node = max(node.children, key=lambda n: n.UCB1())
        return self.Expand(node)

    def Expand(self, node):
        actions = range(self.env.action_space.n)
        for action in actions:
            next_state, reward, done, _, _ = self.env.step(action)
            # Only add child if the state is not terminal
            if not done:
                child_node = node.AddChild(next_state)
        return node  # Return the expanded node

    def Simulate(self, state):
        total_reward = 0
        self.env.reset()
        self.env.env.s = state  # Set the current state
        done = False
        while not done:
            action = self.env.action_space.sample()  # Random action
            next_state, reward, done, _, _ = self.env.step(action)
            total_reward += reward
        return total_reward

    def Backpropagate(self, node, reward):
        while node is not None:
            node.visits += 1
            node.wins += reward
            node = node.parent

def RunMCTS(envName, max_iterations):
    env = gym.make(envName)
    initial_state = env.reset()
    root = Node(initial_state)
    mcts = MCTS(env, max_iterations)
    mcts.Search(root)

    # Find the best action based on the visits
    if root.children:
        best_node = max(root.children, key=lambda n: n.visits)
        return best_node.state
    else:
        print("No children found, cannot determine the best action.")
        return None

result = RunMCTS("FrozenLake-v1", 1000)
print("Estado final alcanzado:", result)

Estado final alcanzado: 0


#### Implementación de Dyna-Q+

In [110]:
class DynaQPlus:
    def __init__(self, env, numEpisodes=1000, alpha=0.1, gamma=0.99, epsilon=0.1, planningSteps=10):
        self.env = env
        self.numEpisodes = numEpisodes
        self.alpha = alpha  # Learning rate
        self.gamma = gamma  # Discount factor
        self.epsilon = epsilon  # Exploration probability
        self.planningSteps = planningSteps  # Number of planning steps
        self.qTable = np.zeros((env.observation_space.n, env.action_space.n))  # Q-table
        self.model = {}  # Model for storing state transitions

    def ChooseAction(self, state):
        if random.random() < self.epsilon:
            return self.env.action_space.sample()  # Explore
        else:
            return np.argmax(self.qTable[state])  # Exploit

    def UpdateModel(self, state, action, reward, nextState):
        self.model[(state, action)] = (reward, nextState)

    def Plan(self):
        for _ in range(self.planningSteps):
            # Sample a random state-action pair from the model
            stateActionPair = random.choice(list(self.model.keys()))
            state, action = stateActionPair
            reward, nextState = self.model[stateActionPair]
            # Update Q-value using the model
            self.qTable[state][action] += self.alpha * (reward + self.gamma * np.max(self.qTable[nextState]) - self.qTable[state][action])

    def Train(self):
        for episode in range(self.numEpisodes):
            state = self.env.reset()  # Reset environment
            if isinstance(state, tuple):  # Check if state is a tuple
                state = state[0]  # Get the first element (the state)
            state = int(state)  # Ensure state is an integer index
            done = False
            while not done:
                action = self.ChooseAction(state)  # Choose action
                nextState, reward, done, _, _ = self.env.step(action)  # Take action
                nextState = int(nextState)  # Ensure nextState is an integer index

                # Update Q-value using real experience
                self.qTable[state][action] += self.alpha * (reward + self.gamma * np.max(self.qTable[nextState]) - self.qTable[state][action])
                # Update model
                self.UpdateModel(state, action, reward, nextState)
                # Plan using the model
                self.Plan()
                state = nextState  # Transition to the next state

def RunDynaQPlus(envName, numEpisodes):
    env = gym.make(envName)
    agent = DynaQPlus(env, numEpisodes=numEpisodes)
    agent.Train()

    # Evaluate the learned policy
    totalRewards = 0
    for _ in range(100):  # Run 100 episodes to evaluate
        state = env.reset()  # Reset environment
        if isinstance(state, tuple):  # Check if state is a tuple
            state = state[0]  # Get the first element (the state)
        state = int(state)  # Ensure state is an integer index
        done = False
        while not done:
            action = np.argmax(agent.qTable[state])  # Choose the best action
            nextState, reward, done, _, _ = env.step(action)
            nextState = int(nextState)  # Ensure nextState is an integer index
            totalRewards += reward
            state = nextState
    
    print("Promedio de recompensas:", totalRewards / 100)

RunDynaQPlus('FrozenLake-v1', 1000)


Promedio de recompensas: 0.0
